<img src='https://user-images.githubusercontent.com/16665629/36117115-329c6220-1041-11e8-98d3-7cd1ce05503c.jpeg'
style="float:right;margin:0 10px 0px 0;width: 150px;">

# Compare observed and theoretical statistical distributions

In [1]:
models_compare = {
                    'meerkathi-pipeline_1-pybdsm.lsm.html': 'meerkathi-pipeline_final-pybdsm.lsm.html',
                    'meerkathi-pipeline_2-pybdsm.lsm.html': 'meerkathi-pipeline_1-pybdsm.lsm.html',
                    'meerkathi-pipeline_3-pybdsm-combined.lsm.html': 'meerkathi-pipeline_2-pybdsm.lsm.html',
                    'meerkathi-pipeline_4-pybdsm-combined.lsm.html': 'meerkathi-pipeline_3-pybdsm-combined.lsm.html',
                    'meerkathi-pipeline_final-pybdsm.lsm.html': 'meerkathi-pipeline_4-pybdsm-combined.lsm.html',
                 }
models_dr = {
                'meerkathi-pipeline_1-pybdsm.lsm.html': 'meerkathi-pipeline_1-MFS-image.fits',
                'meerkathi-pipeline_2-pybdsm.lsm.html':'meerkathi-pipeline_2-MFS-image.fits',
                'meerkathi-pipeline_3-pybdsm-combined.lsm.html': 'meerkathi-pipeline_3-MFS-image.fits',
                'meerkathi-pipeline_4-pybdsm-combined.lsm.html' : 'meerkathi-pipeline_4-MFS-image.fits',
                'meerkathi-pipeline_final-pybdsm.lsm.html': 'meerkathi-pipeline_5-MFS-image.fits',
            }
residuals2 = [
             'meerkathi-pipeline_1-MFS-image.fits',
             'meerkathi-pipeline_2-MFS-image.fits',
             'meerkathi-pipeline_3-MFS-image.fits',
             'meerkathi-pipeline_4-MFS-image.fits',
             'meerkathi-pipeline_5-MFS-image.fits',
            ]
residuals1 = [
             'meerkathi-pipeline_1-MFS-residual.fits',
             'meerkathi-pipeline_2-MFS-residual.fits',
             'meerkathi-pipeline_3-MFS-residual.fits',
             'meerkathi-pipeline_4-MFS-residual.fits',
             'meerkathi-pipeline_5-MFS-residual.fits',
            ]
tolerance = 0.0001 # area factor around the source

In [2]:
from ipywidgets import widgets, interact
from IPython.display import Javascript, display
from IPython.display import HTML

def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+5)'))

directory = widgets.Text(
    value='input',
    placeholder='input',
    description='Input directory:',
    disabled=False
)

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)
display(directory)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

Button(description=u'Configure', style=ButtonStyle())

Text(value=u'input', description=u'Input directory:', placeholder=u'input')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import glob
import combat
import matplotlib
import numpy as np
from plotly import tools
import plotly.graph_objs as go
from plotly import offline as py
from collections import OrderedDict
from scipy.stats import norm, linregress
from sklearn.metrics import r2_score, mean_squared_error
from plotly.graph_objs import Layout, Figure, XAxis, YAxis
from aimfast.aimfast import residual_image_stats, image_dynamic_range, model_dynamic_range
py.init_notebook_mode(connected=True)

In [9]:
ARCSEC_DEG = 1.0/(60*60)
results = combat.property_results(models=models_compare, tolerance=tolerance, input=directory.value)

Loading input/out5/meerkathi-pipeline_4-pybdsm-combined.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_3-pybdsm-combined.lsm.html: Tigger sky model
Number of sources recovered: 50
Loading input/out5/meerkathi-pipeline_3-pybdsm-combined.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_2-pybdsm.lsm.html: Tigger sky model
Number of sources recovered: 24
Loading input/out5/meerkathi-pipeline_2-pybdsm.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_1-pybdsm.lsm.html: Tigger sky model
Number of sources recovered: 21
Loading input/out5/meerkathi-pipeline_1-pybdsm.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_final-pybdsm.lsm.html: Tigger sky model
Number of sources recovered: 22
Loading input/out5/meerkathi-pipeline_final-pybdsm.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_4-pybdsm-combined.lsm.html: Tigger sky model
Number of sources recovered: 50


In [10]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+17)'))

button = widgets.Button(
    description="PLOT"
)
button.on_click(run_all)
_scaley = widgets.Checkbox(
    value=False,
    description='Same Y-scale',
    disabled=False
)
_scalex = widgets.Checkbox(
    value=False,
    description='Same X-scale',
    disabled=False
)
_num_bins = widgets.IntSlider(
    value=50,
    min=0,
    max=1000,
    step=1,
    description='Number of bins:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [11]:
display(button)

Button(description=u'PLOT', style=ButtonStyle())

<IPython.core.display.Javascript object>

## Photometric Measurements

In [12]:
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
for input_model, output_model in models_compare.items():
    counter+=1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    LIMIT = 0
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    I_min_max += flux_out_data
    err_I_min_max += flux_out_err_data
    flux_R_score = r2_score(flux_in_data, flux_out_data)
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    reg = linregress(flux_in_data, flux_in_data)
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    n, bins, patches = matplotlib.pyplot.hist(I_out_in, _num_bins.value, normed=False, facecolor='blue',
                                           orientation='horizontal', alpha=0.5)
    (mu, sigma) = norm.fit(I_out_in)
    peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
    print("Model: {:s} \n{:s} \nSlope: {:.4f} | Mean Squared Error: {:.2E} | R2: {:.4f} "
          "Intercept: {:.4f}\n{:s}".format(heading, '-'*45, reg.slope, flux_MSE, flux_R_score,
                                           reg.intercept, '='*75))
    counter += 1
matplotlib.pylab.close()

Model: meerkathi-pipeline_3-pybdsm-combined 
--------------------------------------------- 
Slope: 1.0000 | Mean Squared Error: 6.25E-36 | R2: 1.0000 Intercept: 0.0000
Model: meerkathi-pipeline_2-pybdsm 
--------------------------------------------- 
Slope: 1.0000 | Mean Squared Error: 1.28E-35 | R2: 1.0000 Intercept: 0.0000
Model: meerkathi-pipeline_1-pybdsm 
--------------------------------------------- 
Slope: 1.0000 | Mean Squared Error: 1.67E-07 | R2: 1.0000 Intercept: 0.0000
Model: meerkathi-pipeline_final-pybdsm 
--------------------------------------------- 
Slope: 1.0000 | Mean Squared Error: 1.61E-07 | R2: 1.0000 Intercept: 0.0000
Model: meerkathi-pipeline_4-pybdsm-combined 
--------------------------------------------- 
Slope: 1.0000 | Mean Squared Error: 6.25E-36 | R2: 1.0000 Intercept: 0.0000


In [13]:
data = []
im_titles =[]
flux_im = dict()
BINS = _num_bins.value
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
    im_titles.append('<b>{:s} flux density</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i = -1
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
for input_model, output_model in models_compare.items():
    i += 1
    counter += 1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    LIMIT = 0
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    flux_R_score = r2_score(flux_in_data, flux_out_data)
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    fig.append_trace(go.Scatter(x=np.array([flux_in_data[0], flux_in_data[-1]]), showlegend=False,
                                y=np.array([flux_in_data[0], flux_in_data[-1]]), mode = 'line'), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(flux_in_data), y=np.array(flux_out_data),
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '%s flux_ratio' % heading,
                                marker = dict(color = phase_center_dist, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', titlefont=dict(size=16), x = 1.0)
                                             ) if i == 0 else dict(color = phase_center_dist, colorscale='Jet',
                                                                   reversescale=False),
                                error_y=dict(type='data', array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(x=0.8,y=1.0),)
    fig['layout'].update(
        {'yaxis{}'.format(counter):YAxis(title=u'I_out (Jy)',gridcolor='rgb(255,255,255)',
        #range=[1,10],
        tickfont=dict(size=15),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter):XAxis(title='I_in (Jy)', position=0.0,
                                                            titlefont=dict(size=17),
                                                            overlaying='x')})
    fig['layout']['annotations'].update({ 'font':{'size': 10}})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Astrometric Measurements

In [20]:
counter = 0
pos_min_max = []
DEC_off_min_max = []
RA_off_min_max = []
for input_model, output_model in models_compare.items():
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    pi,sin,cos = np.pi,np.sin,np.cos
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    theta = np.linspace(0,2*pi,45)
    x1 = r1*cos(theta)
    y1 = r2*sin(theta)
    print("{:s}\n1 sigma = {:f}\n{:s}".format(heading, np.sqrt(r1*r2), 20*'='))

meerkathi-pipeline_3-pybdsm-combined
1 sigma = 0.000000
meerkathi-pipeline_2-pybdsm
1 sigma = 0.000000
meerkathi-pipeline_1-pybdsm
1 sigma = 0.508075
meerkathi-pipeline_final-pybdsm
1 sigma = 0.845079
meerkathi-pipeline_4-pybdsm-combined
1 sigma = 0.000000


In [21]:
data = []
im_titles =[]
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
    im_titles.append('<b>{:s} Position</b>'.format(header.upper()))
    im_titles.append('<b>{:s} Position</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i = -1
counter = 0
for input_model, output_model in models_compare.items():
    i += 1
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    fig.append_trace(go.Scatter(x=np.array(flux_in_data), y=np.array(delta_pos_data),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True,
                                              colorscale='Jet', reversescale=True,
                                              colorbar = dict(title= 'Distance from phase centre (arcsec)',
                                                              titleside ='right', titlefont=dict(size=16), x = 0.4)
                                             ) if i == 0 else dict(color = DELTA_PHASE0, colorscale='Jet',
                                                                   reversescale=True),
                                error_y=dict(type='data', #array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(RA_offset), y=np.array(DEC_offset),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '{:s} flux_ratio'.format(heading),
                                marker = dict(color = flux_out_data, showscale=True, colorscale='Viridis', reversescale=True,
                                              colorbar = dict(title= 'Input Flux (Jy)', titleside ='right',
                                                             titlefont=dict(size=16))
                                             ) if i == 0 else dict(color = flux_out_data, colorscale='Viridis',
                                                                   reversescale=True),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    theta = np.linspace(0,2*pi,45)
    x1 = r1*cos(theta)
    y1 = r2*sin(theta)
    fig.append_trace(go.Scatter(x=x1, y=y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '1σ',
                                text = '1σ ~ {:f}'.format(np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(0, 0, 255)')), i+1, 2)
    fig.append_trace(go.Scatter(x=2*x1, y=2*y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '2σ',
                                text = '2σ ~ {:f}'.format(2*np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(255, 0, 0)')), i+1, 2)
    fig.append_trace(go.Scatter(x=3*x1, y=3*y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '3σ',
                                text = '3σ ~ {:f}'.format(3*np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(255, 255, 0)')), i+1, 2)
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True, x=1.1,y=1)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Delta position [arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
                                           range=axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Declination offset [arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=dec_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input Flux (Jy)',
                                                            titlefont=dict(size=17),
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Right Ascension offset [arcsec]',
                                                              titlefont=dict(size=17),
                                                              range=ra_min_max if _scalex.value else [],
                                                              zeroline=False)})# domain=[0.505, 0.8])})
    fig['layout']['annotations'].update({ 'font':{'size': 10}})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Morphological Measurements

In [22]:
data = []
im_titles =[]
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
    im_titles.append('<b>{:s} Source shape</b>'.format(header.upper()))
    im_titles.append('<b>{:s} Source shape</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.1,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i = -1
counter = 0
for input_model, output_model in models_compare.items():
    i += 1
    counter+=1
    SCALE = []
    SCALE_ERR = []
    flux_in_data = []
    DELTA_PHASE0 = []
    source_labels = []
    MAJ_MIN_angle_in = []
    MAJ_MIN_angle_out = []
    heading = output_model[:-9]
    for n in range(len(results[header]['flux'])):
        MAJ_MIN_angle_out.append(results[header]['shape'][n][0])
        MAJ_MIN_angle_in.append(results[header]['shape'][n][2])
        SCALE.append(results[header]['shape'][n][3])
        SCALE_ERR.append(results[header]['shape'][n][4])
        flux_in_data.append(results[header]['shape'][n][5])
        source_labels.append(results[header]['shape'][n][6])
        DELTA_PHASE0.append(results[header]['position'][n][-3])  
    zipped_props = zip(MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR)
    MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    maj_in = [maj_min_angle_in[0] for maj_min_angle_in in MAJ_MIN_angle_in]
    maj_out = [maj_min_angle_out[0] for maj_min_angle_out in MAJ_MIN_angle_out]
    min_in = [maj_min_angle_in[1] for maj_min_angle_in in MAJ_MIN_angle_in]
    min_out = [maj_min_angle_out[1] for maj_min_angle_out in MAJ_MIN_angle_out]
    angle_offset = [(maj_min_angle_out[2] - maj_min_angle_in[2]) for maj_min_angle_out, maj_min_angle_in in zip(
        MAJ_MIN_angle_out, MAJ_MIN_angle_in)]
    fig.append_trace(go.Scatter(x=maj_in, y=maj_out, mode = 'markers', showlegend=False,
                                text = name_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True, colorscale='Viridis', reversescale=True,
                                              colorbar = dict(title= '', titleside ='right',
                                                             titlefont=dict(size=16))
                                             ) if i == 0 else dict(color = flux_in_data, colorscale='Viridis',
                                                                   reversescale=True),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=min_in, y=min_out,
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True,
                                              colorscale='Viridis', reversescale=True,
                                              colorbar = dict(title= 'Distance from phase center (arcsec)',
                                                              titleside ='right',
                                                              titlefont=dict(size=16))
                                             ) if i == 0 else dict(color = DELTA_PHASE0, colorscale='Viridis',
                                                                   reversescale=True),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Output major axis[arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=14),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Ouput minor axis[arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=14),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input major axis[arcsec]',
                                                            titlefont=dict(size=14),
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Input minor axis[arcsec]',
                                                              titlefont=dict(size=14),
                                                              zeroline=False)})# domain=[0.505, 0.8])}
    fig['layout']['annotations'].update({'font':{'size': 12}})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Dynamic Ranges

In [24]:
beam_factor = 2.0
DRs = OrderedDict()
im_titles = []
dir = directory.value
for m, r in models_dr.items():
    DRs[r[:20]] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), beam_factor)
zipped_props = zip(DRs.keys(), DRs.values())
ims, values = zip(*sorted(zipped_props, key=lambda x: x[0]))
drs, names = [], []
for val in values:
    drs.append(val[0])
    names.append("DR=({:f},{:f})".format(val[1], val[-1]))
fig = tools.make_subplots(rows=1, cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15)
fig.append_trace(go.Scatter(x=ims, showlegend=False, text = names,
                            y=np.array(drs), mode = 'marker'), 1, 1)
fig['layout'].update(title='<b>SNR with self-cal iterations</b>', height=900, width=900,
                     paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                     legend=dict(x=0.8,y=1.0))
fig['layout'].update(
    {'yaxis{}'.format(1):YAxis(title=u'Dynamic Range',gridcolor='rgb(255,255,255)',
        tickfont=dict(size=15),
        titlefont=dict(size=17),
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
fig['layout'].update({'xaxis{}'.format(1):XAxis(title='iterations', position=0.0,
                                                        titlefont=dict(size=17),
                                                        overlaying='x')})
fig['layout']['titlefont'].update({'size':20})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

Loading input/out5/meerkathi-pipeline_4-pybdsm-combined.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_3-pybdsm-combined.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_2-pybdsm.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_1-pybdsm.lsm.html: Tigger sky model
Loading input/out5/meerkathi-pipeline_final-pybdsm.lsm.html: Tigger sky model


## Residual Statistics

In [25]:
Res = OrderedDict()
for res in residuals1:
    Res[res] = residual_image_stats('{:s}/{:s}'.format(directory.value, res))

im_titles = [
               'MEAN',
               'STANDARD DEVIATION',
               'SKEWNESS',
               'KURTOSIS'
            ]
    
fig = tools.make_subplots(rows=4, cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,
                          subplot_titles=im_titles)
    
names = []
mean = []
std = []
skew = []
kurt = []
for name, stats in Res.items():
    names.append(name[:-14])
    mean.append(stats['MEAN'])
    std.append(stats['STDDev'])
    skew.append(stats['SKEW'])
    kurt.append(stats['KURT'])
zipped_props = zip(names, mean, std)
names, mean, std = zip(*sorted(zipped_props, key=lambda x: x[0]))
    
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(mean), mode = 'marker'), 1, 1)
fig['layout'].update(title='', height=900, width=900,
                     paper_bgcolor='rgb(255,255,255)', plot_bgcolor='rgb(229,229,229)',
                         legend=dict(x=0.8,y=1.0),)
fig['layout'].update(
    {'yaxis{}'.format(1):YAxis(title=u'Residual mean (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(1):XAxis(title='iterations', position=0.0,
                                                titlefont=dict(size=17),
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(std), mode = 'marker'), 2, 1)
fig['layout'].update(
    {'yaxis{}'.format(2):YAxis(title=u'Residual sigma (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(2):XAxis(title='iterations', position=0.0,
                                                titlefont=dict(size=17),
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(skew), mode = 'marker'), 3, 1)
fig['layout'].update(
    {'yaxis{}'.format(3):YAxis(title=u'Residual skewness (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(3):XAxis(title='iterations', position=0.0,
                                                titlefont=dict(size=17),
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(kurt), mode = 'marker'), 4, 1)
fig['layout'].update(
    {'yaxis{}'.format(4):YAxis(title=u'Residual kurtosis',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(4):XAxis(title='iterations', position=0.0,
                                                titlefont=dict(size=17),
                                                overlaying='x')})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Source meta-data

In [26]:
src_nm = widgets.Text(
    value='',
    placeholder='ay040',
    description='Source Name:',
    disabled=False
)
display(src_nm)

Text(value=u'', description=u'Source Name:', placeholder=u'ay040')

In [27]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Search")
button.on_click(run_all)
display(button)

Button(description=u'Search', style=ButtonStyle())